In [ ]:
# @title
# Permissão para acessar o drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title
# Instalação do pacote Ultralytics
!pip install ultralytics

In [ ]:
# @title
# Importação das bibliotecas
import cv2 as cv
from glob import glob
import os, sys, re
from ultralytics import YOLO
import math

In [ ]:
# Referenciamos o caminho do arquivo mkv
video_file_path = "/content/drive/MyDrive/ELT578/Projeto_Final/Videos/Camera_16_20231016-104855.mkv"

In [ ]:
# @title
# Convertemos o arquivo de vídeo para o formato mp4
start_time = "00:00:00"  # Substitua pelo tempo de início desejado
end_time = "00:01:00"    # Substitua pelo tempo de término desejado

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -n -i "$inputFile" -c copy -strict -2 "$outputPath"/temp.mkv
!ffmpeg -hide_banner -n -i "$outputPath"/temp.mkv -c copy -strict -2 "$outputPath"/"$fileName".mp4

Input #0, matroska,webm, from '/content/drive/MyDrive/ELT578/Projeto_Final/Videos/Camera_16_20231016-104855.mkv':
  Metadata:
    title           : Media Presentation
    ENCODER         : Lavf58.26.101
  Duration: 00:04:00.42, start: 180.308000, bitrate: 112 kb/s
  Stream #0:0: Video: hevc (Main), yuv420p(tv), 1920x1080, 30.30 fps, 29.92 tbr, 1k tbn, 30 tbc (default)
    Metadata:
      DURATION        : 00:04:00.422000000
Output #0, matroska, to '/content/drive/MyDrive/ELT578/Projeto_Final/Videos//temp.mkv':
  Metadata:
    title           : Media Presentation
    encoder         : Lavf58.76.100
  Stream #0:0: Video: hevc (Main), yuv420p(tv), 1920x1080, q=2-31, 30.30 fps, 29.92 tbr, 1k tbn, 1k tbc (default)
    Metadata:
      DURATION        : 00:04:00.422000000
Stream mapping:
  Stream #0:0 -> #0:0 (copy)
Press [q] to stop, [?] for help
frame= 1800 fps=0.0 q=-1.0 Lsize=    3306kB time=00:01:00.08 bitrate= 450.8kbits/s speed=3.35e+03x    
video:3293kB audio:0kB subtitle:0kB other st

In [ ]:
# @title
# Carregamos os vídeos no formato mp4 em uma variável
videos = glob('/content/drive/MyDrive/ELT578/Projeto_Final/Videos/*.mp4')

In [ ]:
# @title
# Definição do modelo
model_pretrained = YOLO('yolov8n.pt')

In [ ]:
# @title
###VERSÃO 4###
###EXIBE CLS & CONF###

# read video by index
video = cv.VideoCapture(videos[0])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter('/content/drive/MyDrive/ELT578/Projeto_Final/Videos/teste_1.mp4',\
                     fourcc, 20.0, size)

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus",
              "train", "truck", "boat", "traffic light", "fire hydrant",
              "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
              "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase",
              "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket",
              "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
              "banana", "apple", "sandwich", "orange", "broccoli", "carrot",
              "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant",
              "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
              "remote", "keyboard", "cell phone", "microwave", "oven", "toaster",
              "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# Dicionário para rastrear IDs únicos por tipo de veículo
unique_ids = {'car': set(), 'bus': set(), 'motorbike': set(), 'truck': set()}

# Mapeamento de códigos de classe para nomes de classe
class_names = {2: 'car', 5: 'bus', 3: 'motorbike', 7: 'truck'}

# read frames
ret = True

while ret:
    ret, frame = video.read()

    if ret:
        # detect & track objects
        results = model_pretrained.track(frame, persist=True)

        for detection in results[0]:
            boxes = detection.boxes
            for box in boxes:
                # Bounding Box
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                # Confidence
                conf = math.ceil((box.conf[0] * 100)) / 100

                # Class Name
                cls = int(box.cls[0])
                current_class = classNames[cls]

                # id Number
                if box.id is not None:
                        vehicle_id = int(box.id[0])

                # Adiciona o ID ao conjunto correspondente se ainda não estiver presente
                if cls in class_names:
                        unique_ids[class_names[cls]].add(vehicle_id)


                # Verifica se a classe é relevante e a confiança suficiente
                if current_class in class_names.values() and conf > 0.3:
                    cv.putText(frame, f'{current_class} {conf}',
                              (max(0, x1), max(35, y1)),
                               fontFace=cv.FONT_HERSHEY_SIMPLEX,
                               fontScale=0.6, color=(255, 255, 255),
                               thickness=2)


        # Adiciona texto com a quantidade total de veículos por tipo
        y_offset = frame_height // 2 + 30
        for class_code, class_name in class_names.items():
            if class_name in unique_ids:
                text = f'{class_name}s: {len(unique_ids[class_name])}'
                cv.putText(frame, text, (10, y_offset),
                           cv.FONT_HERSHEY_SIMPLEX, 1, (50, 50, 255), 2)
                y_offset += 30

        # save video
        out.write(frame)

out.release()
video.release()





A saída de streaming foi truncada nas últimas 5000 linhas.
0: 384x640 1 car, 2 buss, 1 fire hydrant, 7.0ms
Speed: 2.5ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 1 fire hydrant, 6.8ms
Speed: 2.2ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 6.9ms
Speed: 2.6ms preprocess, 6.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 7.1ms
Speed: 2.2ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 6.8ms
Speed: 2.2ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 6.8ms
Speed: 2.2ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 6.7ms
Speed: 2.2ms preprocess, 6.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384

In [ ]:
# @title
# Compressão do arquivo de saída

# Input video path
save_path = '/content/drive/MyDrive/ELT578/Projeto_Final/Videos/teste_1.mp4'

# Compressed video path
compressed_path = '/content/drive/MyDrive/ELT578/Projeto_Final/Videos/resultado_100.mp4'

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")



0

In [ ]:
# @title
# Instalação do pacote kora
# Útil para exibição de vídeo no google colab
!pip install -U kora

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00


In [ ]:
# @title
# Alteramos o diretório de trabalho
%cd /content/drive/MyDrive/ELT578/Projeto_Final/Videos/

/content/drive/MyDrive/ELT578/Projeto_Final/Videos


In [ ]:
# @title
# Exibição do arquivo de saída
from IPython.display import HTML
from kora.drive import upload_public

url = upload_public('resultado_10.mp4')
HTML(f"""<video src={url} width=960 controls/>""")

In [ ]:
# @title
url = upload_public('resultado_20.mp4')
HTML(f"""<video src={url} width=960 controls/>""")